# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
# импортируем необходимые библиотеки

In [2]:
import requests
import pandas as pd
import sqlite3

## Список объектов

In [3]:
# загружаем данные

In [4]:
offset_ = 1
limit_ = 5000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

In [5]:
# Попытка обойти 504 ошибку и скачать 10000 (не получилось) 

In [6]:
'''
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session
'''

"\nimport requests\nfrom requests.adapters import HTTPAdapter\nfrom requests.packages.urllib3.util.retry import Retry\n\n\ndef requests_retry_session(\n    retries=3,\n    backoff_factor=0.3,\n    status_forcelist=(500, 502, 504),\n    session=None,\n):\n    session = session or requests.Session()\n    retry = Retry(\n        total=retries,\n        read=retries,\n        connect=retries,\n        backoff_factor=backoff_factor,\n        status_forcelist=status_forcelist,\n    )\n    adapter = HTTPAdapter(max_retries=retry)\n    session.mount('http://', adapter)\n    session.mount('https://', adapter)\n    return session\n"

In [7]:
'''
offset_ = 1
limit_ = 10000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'


response = requests_retry_session().get(url)
print(response.status_code)

s = requests.Session()
s.auth = ('user', 'pass')
s.headers.update({'x-test': 'true'})


response = requests_retry_session(session=s).get(url)
'''

"\noffset_ = 1\nlimit_ = 10000\nurl = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'\n\n\nresponse = requests_retry_session().get(url)\nprint(response.status_code)\n\ns = requests.Session()\ns.auth = ('user', 'pass')\ns.headers.update({'x-test': 'true'})\n\n\nresponse = requests_retry_session(session=s).get(url)\n"

In [8]:
# res.json()

In [9]:
objects_list = objects_data.get('data').get('list')

# Посмотрим на скачанные данные

In [10]:
df = pd.json_normalize(objects_list)

In [11]:
df.head()

,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13383,72,г Тюмень,4,20,68,2022-06-30,6289.98,1,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
1,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
2,13398,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,4812.98,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
3,13399,72,"г Тюмень, ул Профсоюзная, д. 56",1,1,0,2024-12-31,0.00,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
4,23682,47,д Агалатово,10,10,54,2022-06-30,2748.15,1,0,...,Общество с ограниченной ответственностью,ООО,5.562608e+09,7806536079,"Жилой комплекс ""84 высота""",NaN,NaN,NaN,NaN,NaN


In [12]:
# в датасете 5000 наблюдений, большая часть переменных заполнена полностью или почти полностью
# есть 3 переменных, в которых заполнено 80% значений, 1 переменная с 500 значениями,
# а также 4 переменных, в которых данные есть только в 50 строках
# переменные, в которых есть пропуски, потребуют предварительной обработки перед дальнейшим анализом

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 35 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   objId                                      5000 non-null   int64  
 1   rpdRegionCd                                5000 non-null   int64  
 2   objAddr                                    4975 non-null   object 
 3   objFloorMin                                5000 non-null   int64  
 4   objFloorMax                                5000 non-null   int64  
 5   objElemLivingCnt                           5000 non-null   int64  
 6   objReady100PercDt                          5000 non-null   object 
 7   objSquareLiving                            5000 non-null   float64
 8   objCompensFundFlg                          5000 non-null   int64  
 9   objProblemFlg                              5000 non-null   int64  
 10  latitude                

In [14]:
# некоторые переменные также необходимо преобразовать:
# (перечислены не все возможные проблемы, а только несколько кейсов)
#
# например, тип rpdRegionCd - число, таким образом если мы не поменяем тип на str, мы можем увидеть "зависимость":
# напр., между значением цены на квартиру и номером региона, хотя по смыслу эта переменная категориальная
#
# также есть переменная со значением "NONE" - это пропуски, но python не может их определить (problemFlag)  

In [15]:
df.describe()

,objId,rpdRegionCd,objFloorMin,objFloorMax,objElemLivingCnt,objSquareLiving,objCompensFundFlg,objProblemFlg,latitude,longitude,objGuarantyEscrowFlg,residentialBuildings,pdId,objStatus,developer.devId,developer.orgForm.id,developer.companyGroup,developer.bankruptStage.bankruptStageCd
count,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,4989.000000,4989.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,3.899000e+03,53.000000
mean,34829.523600,48.975600,10.40580,12.378400,183.429800,9144.164200,0.170400,0.0,53.973484,51.293056,0.757600,0.897600,317857.808200,0.0,6208.513400,4.939600,3.202666e+09,4.698113
std,10812.148733,22.404691,7.98831,8.678749,232.338653,11451.939214,0.376021,0.0,5.000651,26.025691,0.428578,0.303204,198703.078453,0.0,4678.830632,10.817641,2.740626e+09,0.991985
min,851.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,41.419900,19.903300,0.000000,0.000000,172.000000,0.0,10.000000,1.000000,2.970000e+02,4.000000
25%,30040.750000,29.000000,3.00000,5.000000,31.000000,1814.490000,0.000000,0.0,52.224100,37.249100,1.000000,1.000000,90394.750000,0.0,1704.000000,4.000000,4.516860e+08,4.000000
50%,38630.500000,50.000000,9.00000,11.000000,112.000000,6027.600000,0.000000,0.0,55.400200,39.600900,1.000000,1.000000,394693.000000,0.0,6320.000000,4.000000,2.488380e+09,4.000000
75%,42847.250000,71.000000,16.00000,18.000000,252.000000,12133.150000,0.000000,0.0,56.799900,60.496700,1.000000,1.000000,495503.000000,0.0,10856.000000,4.000000,5.695252e+09,6.000000
max,45983.000000,92.000000,76.00000,76.000000,2676.000000,142978.700000,1.000000,0.0,66.866000,142.743100,1.000000,1.000000,586599.000000,0.0,14022.000000,124.000000,1.023909e+10,7.000000


In [16]:
df.describe(include = 'object')

,objAddr,objReady100PercDt,problemFlag,siteStatus,rpdNum,buildType,developer.shortName,developer.fullName,developer.groupName,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
count,4975,5000,5000,5000,5000,5000,5000,5000,3899,5000,5000,5000,3893,490,53,53,53
unique,4132,48,1,1,3022,2,1625,1631,766,13,11,1721,1988,150,3,3,21
top,г Санкт-Петербург,2022-12-31,NONE,Строящиеся,77-001887,Жилое,СЗ ГЕЛЕОН СТРОЙ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ГЕЛЕОН СТРОЙ,ЛСР,Общество с ограниченной ответственностью,ООО,5408029742,Цветной город,Цветной город,Сообщение о судебном акте. О признании должник...,http://www.consultant.ru/document/cons_doc_LAW...,21-06-2019 17:49
freq,93,1019,5000,5000,117,4488,148,148,172,4539,4540,148,61,67,35,35,14


# Сохраним df в различных форматах

In [17]:
df.to_csv('HW1_domrf_5k.csv', sep=';', index = False)

In [18]:
df.to_excel('HW1_domrf_5k.xlsx', index = False)

In [19]:
df.to_pickle('HW1_domrf_5k.p')

In [20]:
# проверим, что получилось

In [21]:
df1 = pd.read_csv('HW1_domrf_5k.csv', sep=';')
df1.head(3)

,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13383,72,г Тюмень,4,20,68,2022-06-30,6289.98,1,0,...,Акционерное общество,АО,699421001.0,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
1,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,0,...,Акционерное общество,АО,699421001.0,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
2,13398,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,4812.98,0,0,...,Акционерное общество,АО,699421001.0,7701651356,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df2 = pd.read_excel('HW1_domrf_5k.xlsx')
df2.tail(3)

,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
4997,39739,75,"г Чита, микрорайон 3, д. 20",17,17,156,2023-03-31,7756.00,0,0,...,Общество с ограниченной ответственностью,ООО,5.581246e+09,7536050334,Дом №20 в 3 микрорайоне г. Читы,NaN,NaN,NaN,NaN,NaN
4998,45796,29,"г Северодвинск, в районе улицы Мира, д. 5",10,10,208,2024-12-31,8435.06,0,0,...,Общество с ограниченной ответственностью,ООО,NaN,2902088977,ЖК Аквилон OSTROV,NaN,NaN,NaN,NaN,NaN
4999,45974,77,"г Москва, район Лефортово, улица Золоторожский...",1,27,192,2025-09-30,12566.20,0,0,...,Общество с ограниченной ответственностью,ООО,4.302780e+08,7722489986,СИМВОЛ,NaN,NaN,NaN,NaN,NaN


In [23]:
df3 = pd.read_pickle('HW1_domrf_5k.p')
df3.head(3)

,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13383,72,г Тюмень,4,20,68,2022-06-30,6289.98,1,0,...,Акционерное общество,АО,699421001.0,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
1,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,0,...,Акционерное общество,АО,699421001.0,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
2,13398,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,4812.98,0,0,...,Акционерное общество,АО,699421001.0,7701651356,NaN,NaN,NaN,NaN,NaN,NaN


# Сохраним данные в БД

In [24]:
conn = sqlite3.connect('test_database')

In [25]:
df.to_sql('dom_RF', conn, if_exists='replace', index = False)
conn.commit()


In [26]:
df4 = pd.read_sql('select * from dom_RF', con = conn)
df4.head(3)

,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13383,72,г Тюмень,4,20,68,2022-06-30,6289.98,1,0,...,Акционерное общество,АО,699421001.0,7701651356,None,None,NaN,None,None,None
1,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,0,...,Акционерное общество,АО,699421001.0,7701651356,None,None,NaN,None,None,None
2,13398,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,4812.98,0,0,...,Акционерное общество,АО,699421001.0,7701651356,None,None,NaN,None,None,None


In [27]:
conn.close()